In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import sagemaker.amazon.common as smac

In [20]:
 # Read CSV file in Panda dataframe
bucket='sagemaker-pratlda'
data_key = 'all_tickets.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
df= pd.read_csv(data_location)


In [21]:
 df.head

<bound method NDFrame.head of                                                   title  \
0                                                   NaN   
1                                  connection with icon   
2                                  work experience user   
3                                requesting for meeting   
4                 reset passwords for external accounts   
...                                                 ...   
48544  git space for a project issues with adding users   
48545                                             error   
48546                                 connection issues   
48547                                error cube reports   
48548                         running out on extensions   

                                                    body  ticket_type  \
0      hi since recruiter lead permission approve req...            1   
1      icon dear please setup icon per icon engineers...            1   
2      work experience user hi work experi

In [22]:
df['num_words'] = df.body.apply(lambda x : len(x.split()))

In [23]:
bins=[0,50,75, np.inf]
df['bins']=pd.cut(df.num_words, bins=[0,100,300,500,800, np.inf], labels=['0-100', '100-300', '300-500','500-800' ,'>800'])

In [24]:
word_distribution = df.groupby('bins').size().reset_index().rename(columns={0:'counts'})

In [25]:
conda install -c conda-forge wordcloud

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.4
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs:
    - wordcloud


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aiobotocore-1.2.2          |     pyhd8ed1ab_0          40 KB  conda-forge
    aiohttp-3.7.3              |   py36h8f6f2f9_2         629 KB  conda-forge
    astroid-2.5                |   py36h5fab9bb_1         297 KB  conda-forge
    botocore-1.19.52           |     pyhd8ed1ab_0         4.4 MB  conda-forge
    lxml-4.6.2                 |   py36

In [28]:
df.head()

,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact,num_words,bins
0,NaN,hi since recruiter lead permission approve req...,1,4,2,21,71,3,4,12,0-100
1,connection with icon,icon dear please setup icon per icon engineers...,1,6,22,7,26,3,4,15,0-100
2,work experience user,work experience user hi work experience studen...,1,5,13,7,32,3,4,16,0-100
3,requesting for meeting,requesting meeting hi please help follow equip...,1,5,13,7,32,3,4,11,0-100
4,reset passwords for external accounts,re expire days hi ask help update passwords co...,1,4,2,76,4,3,4,140,100-300


In [29]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import numpy as np
import pandas as pd
import re
import warnings

#Visualisation
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from IPython.display import display
#from mpl_toolkits.basemap import Basemap
from wordcloud import WordCloud, STOPWORDS

#nltk
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize

In [30]:
df['body'] = df['body'].astype(str)

In [31]:
data_description = df.body.values

In [32]:
type(data_description)

numpy.ndarray

In [ ]:
#cv = # Initialize a CountVectorizer object: count_vectorizer
count_vec = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 1), max_df=1, min_df=1, max_features=None)

# convert text data into term-frequency matrix
data_description = count_vec.fit_transform(data_description)




tfidf_transformer = TfidfTransformer()

# convert term-frequency matrix into tf-idf
tfidf_matrix = tfidf_transformer.fit_transform(data_description)

# create dictionary to find a tfidf word each word
word2tfidf = dict(zip(count_vec.get_feature_names(), tfidf_transformer.idf_))

for word, score in word2tfidf.items():
     print(word, score)

In [34]:
term_frequency = count_vec.vocabulary_
type(term_frequency)

dict

In [35]:
sorted_by_idf = np.argsort(tfidf_transformer.idf_)
feature_names = np.array(count_vec.get_feature_names())
print("Features with lowest idf:\n{}".format(
       feature_names[sorted_by_idf[:10]]))
print("\nFeatures with highest idf:\n{}".format(
       feature_names[sorted_by_idf[-10:]]))

Features with lowest idf:
['abbott' 'previews' 'priced' 'pricy' 'prim' 'primate' 'priory' 'prise'
 'privet' 'probed']

Features with highest idf:
['faded' 'fades' 'fads' 'faint' 'fairness' 'fakes' 'fallowed' 'familia'
 'facial' 'zori']


In [37]:
count = nltk.FreqDist(df.body.values)


In [38]:
final_doc = df['body']
all_documents = tuple(final_doc)

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_documents)
#print (tfidf_matrix.shape)
#print(tfidf_matrix)

In [40]:
reindexed_data= df["body"]

data_text = df[['body']]
data_text['index'] = data_text.index
documents=data_text


print(len(documents))
print(documents[:5])



48549
                                                body  index
0  hi since recruiter lead permission approve req...      0
1  icon dear please setup icon per icon engineers...      1
2  work experience user hi work experience studen...      2
3  requesting meeting hi please help follow equip...      3
4  re expire days hi ask help update passwords co...      4


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [41]:
pip install gensim

     |████████████████████████████████| 23.9 MB 18.4 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 8.7 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [42]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk


def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [43]:
from nltk.stem.porter import *
nltk.download('wordnet')

stemmer = PorterStemmer()
stemmer.stem('identified')


doc_sample = documents[documents['index'] == 5].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))


processed_docs = documents['body'].map(preprocess)
processed_docs[:5]

[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


original document: 
['verification', 'warning', 'hi', 'has', 'got', 'attached', 'please', 'addresses', 'best', 'regards', 'monitoring', 'analyst', 'verification', 'warning']


 tokenized and lemmatized document: 
['verif', 'warn', 'attach', 'address', 'best', 'regard', 'monitor', 'analyst', 'verif', 'warn']


0    [recruit, lead, permiss, approv, requisit, mak...
1    [icon, dear, setup, icon, icon, engin, detail,...
2    [work, experi, user, work, experi, student, co...
3    [request, meet, help, follow, equip, cabl, cor...
4    [expir, day, help, updat, password, colleagu, ...
Name: body, dtype: object

In [44]:

dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 approv
1 lead
2 make
3 permiss
4 recruit
5 requisit
6 thank
7 dear
8 detail
9 engin
10 icon


In [45]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[123]

[(0, 1),
 (7, 1),
 (18, 2),
 (26, 1),
 (28, 2),
 (29, 5),
 (35, 11),
 (56, 1),
 (59, 1),
 (71, 6),
 (84, 5),
 (85, 1),
 (89, 1),
 (91, 1),
 (92, 2),
 (99, 3),
 (107, 1),
 (109, 1),
 (124, 1),
 (136, 3),
 (143, 2),
 (145, 1),
 (149, 1),
 (150, 2),
 (156, 16),
 (158, 1),
 (163, 1),
 (190, 2),
 (202, 10),
 (203, 1),
 (215, 1),
 (235, 5),
 (253, 2),
 (263, 1),
 (278, 2),
 (280, 1),
 (282, 16),
 (287, 1),
 (293, 1),
 (313, 1),
 (318, 1),
 (319, 1),
 (324, 1),
 (340, 4),
 (345, 1),
 (357, 1),
 (365, 1),
 (373, 1),
 (376, 1),
 (383, 1),
 (442, 2),
 (458, 1),
 (464, 1),
 (509, 1),
 (571, 1),
 (585, 1),
 (657, 1),
 (716, 1),
 (721, 1),
 (739, 1),
 (740, 1),
 (741, 16),
 (742, 1),
 (743, 1),
 (744, 1),
 (745, 1),
 (746, 1)]

In [46]:
# Preview Bag Of Words for our sample preprocessed document.
bow_doc_123 = bow_corpus[123]
for i in range(len(bow_doc_123)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_123[i][0], 
                                               dictionary[bow_doc_123[i][0]], 
bow_doc_123[i][1]))

Word 0 ("approv") appears 1 time.
Word 7 ("detail") appears 1 time.
Word 18 ("work") appears 2 time.
Word 26 ("request") appears 1 time.
Word 28 ("activ") appears 2 time.
Word 29 ("best") appears 5 time.
Word 35 ("close") appears 11 time.
Word 56 ("issu") appears 1 time.
Word 59 ("log") appears 1 time.
Word 71 ("regard") appears 6 time.
Word 84 ("analyst") appears 5 time.
Word 85 ("attach") appears 1 time.
Word 89 ("anymor") appears 1 time.
Word 91 ("kind") appears 1 time.
Word 92 ("look") appears 2 time.
Word 99 ("card") appears 3 time.
Word 107 ("error") appears 1 time.
Word 109 ("friday") appears 1 time.
Word 124 ("prod") appears 1 time.
Word 136 ("tuesday") appears 3 time.
Word 143 ("absenc") appears 2 time.
Word 145 ("action") appears 1 time.
Word 149 ("countri") appears 1 time.
Word 150 ("date") appears 2 time.
Word 156 ("leav") appears 16 time.
Word 158 ("past") appears 1 time.
Word 163 ("record") appears 1 time.
Word 190 ("hello") appears 2 time.
Word 202 ("assign") appears 10 

In [47]:
#TF-IDF
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.17292971048228067),
 (1, 0.1952915654258252),
 (2, 0.30632481285788143),
 (3, 0.24819355264410153),
 (4, 0.8199977068851184),
 (5, 0.3226845981181564)]


In [48]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)

In [49]:
# Print the results
 
# Here we can assign topic names to each topic list, you can specify the number of words you want to have shown (e.g num_words = 5)
result = lda_model.print_topics(num_topics=5, num_words=5)
print(result)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

[(0, '0.020*"site" + 0.019*"inform" + 0.013*"confluenc" + 0.011*"messag" + 0.011*"juli"'), (1, '0.031*"hello" + 0.027*"regard" + 0.026*"form" + 0.017*"wednesday" + 0.017*"tuesday"'), (2, '0.037*"regard" + 0.027*"hello" + 0.021*"kind" + 0.020*"engin" + 0.017*"order"'), (3, '0.049*"send" + 0.023*"issu" + 0.021*"error" + 0.016*"regard" + 0.014*"hello"'), (4, '0.016*"card" + 0.016*"task" + 0.015*"detail" + 0.013*"request" + 0.013*"assign"')]
Topic: 0 
Words: 0.020*"site" + 0.019*"inform" + 0.013*"confluenc" + 0.011*"messag" + 0.011*"juli" + 0.011*"regard" + 0.010*"regist" + 0.010*"provid" + 0.009*"test" + 0.009*"content"
Topic: 1 
Words: 0.031*"hello" + 0.027*"regard" + 0.026*"form" + 0.017*"wednesday" + 0.017*"tuesday" + 0.016*"leaver" + 0.016*"chang" + 0.014*"approv" + 0.014*"thursday" + 0.014*"juli"
Topic: 2 
Words: 0.037*"regard" + 0.027*"hello" + 0.021*"kind" + 0.020*"engin" + 0.017*"order" + 0.015*"instal" + 0.014*"connect" + 0.013*"assign" + 0.013*"senior" + 0.013*"purchas"
Topic: 3

In [50]:
# put these in a file and name the topics
with open("lda_topics_result.txt", "w") as f:
    for s in result:
        f.write(str(s) +"\n")


In [ ]:
pip install pyLDAvis

In [53]:
# Visualize
import pyLDAvis.gensim

lda_display = pyLDAvis.gensim.prepare(lda_model, bow_corpus,dictionary, sort_topics=True)
pyLDAvis.display(lda_display)